## Imports


In [27]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from scipy.stats import poisson
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
from datetime import datetime
from datetime import date
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import sasoptpy as so
import os

pd.set_option('display.max_columns', 100)

## Fixture Generating Funtions
Generate ticker for visualization and optimization, based on the live fixture info from the Premier League API and any custom fixture timings and probabilities set by the user.

In [28]:
## Generate matchday ticker dataframe, team_fixtures
def generate_ticker(gw_range=None, exclude_teams=None, custom_fixtures=None, extra_fixtures=None, generate_all_dataframes=False):

    # Infer fixture difficulties
    weerogue_ts = pd.read_csv(f'../data/weerogue_ts_naive.csv')
    weerogue_ts['gd'] = round(weerogue_ts['off']-weerogue_ts['def'], 2)
    home_adv = 1.12
    weerogue_ts['h_off'] = round(weerogue_ts['off'] * home_adv,2)
    weerogue_ts['h_def'] = round(weerogue_ts['def'] / home_adv,2)
    weerogue_ts['h_gd'] = weerogue_ts['h_off'] - weerogue_ts['h_def']
    weerogue_ts['a_off'] = round(weerogue_ts['off'] / home_adv,2)
    weerogue_ts['a_def'] = round(weerogue_ts['def'] * home_adv,2)
    weerogue_ts['a_gd'] = weerogue_ts['a_off'] - weerogue_ts['a_def']

    # Get fixtures and team data from pl api
    r = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/')
    fpl_data = r.json()
    team_data = pd.DataFrame(fpl_data['teams'])
    team_data = team_data[['id', 'short_name']].rename(columns={"id": "team_id"})
    team_data = team_data.replace('NFO', 'FOR')
    r = requests.get('https://fantasy.premierleague.com/api/fixtures/')
    fixtures_data = r.json()
    fixtures_data = pd.DataFrame(fixtures_data)
    fixtures_data = fixtures_data.drop('stats', axis=1)
    fixtures_data = fixtures_data[fixtures_data['started'] != True]
    # fixtures_data.to_csv('../data/fixtures_test_all.csv')
    fixtures_data = fixtures_data[fixtures_data['started'] == False]
    fixtures_data['gw'] = fixtures_data['event'].astype(int)
    fixtures_data['datetime'] = fixtures_data['kickoff_time'].astype('datetime64[ns]')
    fixtures_data['date_str'] = fixtures_data['kickoff_time'].str[:10]
    fixtures_data['time_str'] = fixtures_data['kickoff_time'].str[-9:-4]
    fixtures_data = pd.merge(fixtures_data, team_data, left_on='team_a', right_on='team_id', how='left').rename(columns={"short_name": "team_a_name", "team_id": "team_a_id"})
    fixtures_data = pd.merge(fixtures_data, team_data, left_on='team_h', right_on='team_id', how='left').rename(columns={"short_name": "team_h_name", "team_id": "team_h_id"})

    # Add customized fixtures to fixtures table
    fixtures_data.loc[:,'customized'] = False
    fixtures_data['custom_dates'] = [[] for _ in range(len(fixtures_data))]
    fixtures_data['custom_probs'] = [[] for _ in range(len(fixtures_data))]
    if custom_fixtures is not None:
        custom_fixtures['added_to_ticker'] = False
        for index, row in custom_fixtures.iterrows():
            h = custom_fixtures.loc[index,'home_team']
            a = custom_fixtures.loc[index,'away_team']
            listy = fixtures_data.index[(fixtures_data['team_h_name'] == h) & (fixtures_data['team_a_name'] == a)].to_list()
            # if the fixture to be added isn't in the fixtures to be played, add it
            if listy != []:
                i = listy[0]
                fixtures_data.at[i, 'customized'] = True
                fixtures_data.at[i, 'custom_dates'] = custom_fixtures.loc[index,'dates']
                fixtures_data.at[i, 'custom_probs'] = custom_fixtures.loc[index,'probabilities']
                custom_fixtures.at[index, 'added_to_ticker'] = True

    # Drop rows not in gameweek range
    if gw_range is not None:
        mask = fixtures_data['gw'].isin(gw_range)
        fixtures_data = fixtures_data[mask]

    # Generate ticker
    natural_fix_dates = sorted(fixtures_data['date_str'].unique())
    custom_fix_dates = []
    if custom_fixtures is not None:
        for i, x in custom_fixtures.iterrows():
            custom_fix_dates += (custom_fixtures.loc[i, 'dates'])
    unique_dates = sorted(natural_fix_dates + custom_fix_dates)
    unique_dates = sorted(list(set(unique_dates)))
    team_fixtures = team_data.assign(**dict.fromkeys(unique_dates, ''))
    old_date = None
    old_datetime = None
    for index, row in fixtures_data.iterrows():
        new_date = row['date_str']
        new_datetime = row['datetime']
        away_team = row['team_a_name']
        home_team = row['team_h_name'].lower()
        if old_date != new_date or (row['datetime'] == old_datetime and first_fix):
            away_team += '!'
            home_team += '!'
            first_fix = True
        else:
            first_fix = False
        team_fixtures.loc[team_fixtures['short_name'] == row['team_h_name'], row['date_str']] = away_team
        team_fixtures.loc[team_fixtures['short_name'] == row['team_a_name'], row['date_str']] = home_team
        old_date = new_date
        old_datetime = new_datetime
    copied_natural_fixtures = team_fixtures.copy()
    # Add the custom fixtures to the ticker, deleting their 'natural' placement
    # NB: only fixtures that have yet to be played can be added
    if custom_fixtures is not None:
        for index, row in fixtures_data.iterrows():
            if row['customized']:
                team_fixtures.loc[team_fixtures['short_name'] == row['team_h_name'], row['date_str']] = ''
                team_fixtures.loc[team_fixtures['short_name'] == row['team_a_name'], row['date_str']] = ''
                for i, x in enumerate(row['custom_probs']):
                    if x == 1:
                        prob_str = ''
                    elif x == 0:
                        break
                    else:
                        prob_str = '*' + str(int(x*100)) + '%' 
                    date = row['custom_dates'][i]      
                    away_team = row['team_a_name'] + '!' + prob_str
                    home_team = row['team_h_name'].lower() + '!' + prob_str
                    if team_fixtures.loc[team_fixtures['short_name'] == row['team_h_name'], date].to_list()[0] != '':
                        away_team = '\n' + away_team
                    if team_fixtures.loc[team_fixtures['short_name'] == row['team_a_name'], date].to_list()[0] != '':
                        home_team = '\n' + home_team
                    team_fixtures.loc[team_fixtures['short_name'] == row['team_h_name'], date] += away_team
                    team_fixtures.loc[team_fixtures['short_name'] == row['team_a_name'], date] += home_team
        # Add those fixtures which aren't included in the natural fixtures
        if False in custom_fixtures['added_to_ticker'].tolist():
            extra_custom_fixtures = custom_fixtures.loc[custom_fixtures['added_to_ticker'] == False]
            for index, row in extra_custom_fixtures.iterrows():
                for i, x in enumerate(row['probabilities']):
                    if x == 1:
                        prob_str = ''
                    else:
                        prob_str = '*' + str(int(x*100)) + '%' 
                    date = row['dates'][i]      
                    away_team = row['away_team'] + '!' + prob_str
                    home_team = row['home_team'].lower() + '!' + prob_str
                    if team_fixtures.loc[team_fixtures['short_name'] == row.loc['home_team'], date].to_list() != []:
                        if team_fixtures.loc[team_fixtures['short_name'] == row.loc['home_team'], date].to_list()[0] != '':
                            away_team = '\n' + away_team
                    if team_fixtures.loc[team_fixtures['short_name'] == row.loc['away_team'], date].to_list() != []:
                        if team_fixtures.loc[team_fixtures['short_name'] == row.loc['away_team'], date].to_list()[0] != '':
                            home_team = '\n' + home_team
                    prev1 = team_fixtures.loc[team_fixtures['short_name'] == row.loc['home_team'], date] + away_team
                    prev2 = team_fixtures.loc[team_fixtures['short_name'] == row.loc['away_team'], date] + home_team
                    team_fixtures.loc[team_fixtures['short_name'] == row.loc['home_team'], date] = prev1
                    team_fixtures.loc[team_fixtures['short_name'] == row.loc['away_team'], date] = prev2
                    custom_fixtures.loc[index, 'added_to_ticker'] = True

    # Generate matchday mapping to sky gw, fpl gw, date, and day of week
    weekdays = []
    matchdays = []
    sky_gw_list = []
    fpl_gw_list = []
    sky_gw_df = pd.read_csv('../data/sky_gw_starts_2223.csv')
    fpl_gw_df = pd.read_csv('../data/fpl_gw_starts_2223.csv')
    sky_gw_date = sky_gw_df.loc[0,'start_date']
    fpl_gw_date = fpl_gw_df.loc[0,'start_date']
    fpl_gw_index = 0
    sky_gw_index = 0
    # loop through matchdays
    for i, x in enumerate(unique_dates):
        new_day = str(datetime.strptime(unique_dates[i], '%Y-%m-%d').date().weekday())
        weekdays.append(new_day)
        matchdays.append(i+1)
        # while the date of the current matchday is later than that of the proposed sky gw, proceed to the date of the next sky gw
        while x >= sky_gw_date:
            broken = False
            if sky_gw_index > len(sky_gw_df)-1:
                broken = True
                break
            sky_gw_index += 1
            sky_gw_date = sky_gw_df.loc[sky_gw_index-1, 'start_date']
        if broken:
            sky_gw = sky_gw + 1
        else:
            sky_gw = sky_gw_df.loc[sky_gw_index-1, 'gameweek']-1
        sky_gw_list.append(sky_gw)
        # while the date of the current matchday is later than that of the proposed fpl gw, proceed to the date of the next fpl gw
        while x >= fpl_gw_date:
            broken = False
            if fpl_gw_index > len(fpl_gw_df)-1:
                broken = True
                break
            fpl_gw_index += 1
            fpl_gw_date = fpl_gw_df.loc[fpl_gw_index-1, 'start_date']
        if broken:
            fpl_gw = fpl_gw + 1
        else:
            fpl_gw = fpl_gw_df.loc[fpl_gw_index-1, 'gameweek']-1
        fpl_gw_list.append(fpl_gw)
    data = {'unique_dates': unique_dates,
            'weekday': weekdays,
            'matchday': matchdays,
            'sky_gw': sky_gw_list,
            'fpl_gw': fpl_gw_list
            }
    md_map = pd.DataFrame(data)
    date_0 = md_map.loc[0,'unique_dates']
    date_0 = datetime.strptime(date_0, '%Y-%m-%d').date()
    for i, col in md_map.iterrows():
        date_1 = md_map.loc[i,'unique_dates']
        date_1 = datetime.strptime(date_1, '%Y-%m-%d').date()
        delta = int((date_1 - date_0).days)
        md_map.loc[i,'days_elapsed'] = delta

    # Generate dataframes for all possible fixture permutations for stochastic optimization, assuming all fixtures are independent
    if generate_all_dataframes and extra_fixtures is not None:
        uncertain_fixtures = extra_fixtures.drop(extra_fixtures[extra_fixtures.probability == 1].index)
        my_list = uncertain_fixtures.probability.tolist()
        # Assume all fixtures are independent
        n_uncert_fix = len(my_list)
        number_of_permutations = 2**(n_uncert_fix)
        fix_permutation_dict = {}
        for i in range(number_of_permutations):
            fixture_key = f'permutation_{i+1}'
            permutation_string = format(i, f'0{n_uncert_fix}b')
            df = copied_natural_fixtures.copy()
            likelihood = 1
            for i, x in enumerate(permutation_string):
                if bool(int(x)):
                    away_team = uncertain_fixtures.loc[i,'away_team']
                    home_team = uncertain_fixtures.loc[i,'home_team']
                    df.loc[df['short_name'] == home_team, uncertain_fixtures.loc[i,'date']] = away_team
                    df.loc[df['short_name'] == away_team, uncertain_fixtures.loc[i,'date']] = home_team.lower()
                    likelihood = likelihood * uncertain_fixtures.loc[i,'probability']
                else:
                    likelihood = likelihood * (1-uncertain_fixtures.loc[i,'probability'])
            fix_permutation_dict[fixture_key] = {'df': df, 'likelihood': likelihood}
    else:
        fix_permutation_dict = None

    fixtures_df = team_fixtures.copy()

    # Drop excluded teams
    if exclude_teams is not None:
        for i in exclude_teams:
            team_fixtures = team_fixtures[team_fixtures.short_name != i]

    # Add gameweek superheader
    date_to_gw = fixtures_data[['date_str','gw']].drop_duplicates()
    headers = list(team_fixtures.columns.values)
    sky_gw_header = []
    fpl_gw_header = []
    for i in headers:
        if i.startswith('20') == False:
            j = 'sky_gw'
            k = 'fpl_gw'
        else:
            sky_gw = md_map.loc[md_map['unique_dates']==i,'sky_gw'].values[0]
            fpl_gw = md_map.loc[md_map['unique_dates']==i,'fpl_gw'].values[0]
            j = str(sky_gw)
            k = str(fpl_gw)
        sky_gw_header.append(j)
        fpl_gw_header.append(k)
    md_header = []
    for i in headers:
        if i.startswith('20') == False:
            j = 'matchday'
        else:
            md = md_map.loc[md_map['unique_dates']==i,'matchday'].values[0]
            j = str(md)
        md_header.append(j)
    team_fixtures.columns=[fpl_gw_header, sky_gw_header, md_header, headers]


    formatted_fixtures = team_fixtures.copy()
    # Make color map dictionary and function
    color_ts = weerogue_ts[['short_name','h_gd', 'a_gd']].copy()
    min_gd = min(color_ts['h_gd'].values.tolist() + color_ts['a_gd'].values.tolist())*2.3
    max_gd = max(color_ts['h_gd'].values.tolist() + color_ts['a_gd'].values.tolist())#*1.8
    norm = matplotlib.colors.Normalize(vmin=min_gd, vmax=max_gd, clip=True)
    mapper = plt.cm.ScalarMappable(norm=norm, cmap=plt.cm.viridis_r)
    color_ts['h_gd_color'] = color_ts['h_gd'].apply(lambda x: mcolors.to_hex(mapper.to_rgba(x)))
    color_ts['a_gd_color'] = color_ts['a_gd'].apply(lambda x: mcolors.to_hex(mapper.to_rgba(x)))
    h_teams = color_ts['short_name'].values.tolist()
    a_teams = [team.lower() for team in h_teams]
    teams = h_teams + a_teams
    team_gd = color_ts['a_gd_color'].values.tolist() + color_ts['h_gd_color'].values.tolist()
    color_dict = {teams[i]: team_gd[i] for i in range(len(teams))}
    def color_col(col, pattern_map, default=''):
        return np.select(
            [col.str.contains(k, na=False) for k in pattern_map.keys()],
            [f'background-color: {v}' for v in pattern_map.values()],
            default=default
        ).astype(str)
    # Apply styles
    formatted_fixtures = formatted_fixtures.style.apply(color_col,
                                                pattern_map=color_dict
                                                , subset=team_fixtures.columns[2:]
                                                )
    formatted_fixtures = formatted_fixtures.set_table_styles([
                        {'selector': 'th.col_heading', 'props': 'text-align: left;'},
                        {'selector': 'th.col_heading.level0', 'props': 'font-size: 1em;'},
                        {'selector': 'td', 'props': 'text-align: center; font-weight: bold;'},
                    ], overwrite=False)

    return {'formatted_fixtures': formatted_fixtures, 'fixtures_data': fixtures_df, 'matchday_map': md_map, 'fix_permutation_dict': fix_permutation_dict}

## Modelling Functions

Generate dataframe of expected points values for a specified period, based on prior team and player level data. Calls fixture fixture generator function.

In [29]:
def prior_team_data_gen(use_weerogue_ts=1):

    team_data = pd.read_csv('../data/team_data_2023.csv', index_col=0)

    return team_data

def prior_player_data_gen(team_data):
    
    prior_player_data = pd.read_csv('../data/prior_player_data_2023.csv', index_col=0)
    fplreview = pd.read_csv('../data/fplreview.csv')

    # Get gw x_mins from fplreview file, and overwite
    review_gw_list = []
    for element in list(fplreview.columns.values):
        if '_xMins' in element:
            review_gw_list.append(element)
    for element in list(prior_player_data.columns.values):
        if element in review_gw_list:
            prior_player_data = prior_player_data.drop(columns = [element])
    prior_player_data = pd.merge(prior_player_data, fplreview.loc[:,['ID'] + review_gw_list], on=['ID'], how='left')

    # prior_player_data.to_csv('../data/prior_player_data_updated.csv')
    
    return {'prior_player_data':prior_player_data}

def sky_xP_calc(SKY_ID, opp_team, prior_player_data, team_data, xMins, xP_breakdown=False):
    player_data = prior_player_data.loc[prior_player_data['SKY ID'] == SKY_ID].reset_index()
    own_team = player_data.loc[0, 'team_name']
    own_team_data = team_data.loc[team_data['team_name'] == own_team].reset_index()
    opp_data = team_data.loc[team_data['team_name'] == opp_team.upper()].reset_index()
    Pos = player_data.loc[0, 'SKY Pos']
    if opp_team.isupper() == True:
        home_adv = 1.12
    elif opp_team.islower() == True:
        home_adv = 1/1.12
    else:
        home_adv = 1
    if Pos == 'GK':
        k_G = 7
        k_CS = 7
        k_2GC = -1
    elif Pos == 'DEF':
        k_G = 7
        k_CS = 5
        k_2GC = -1
    elif Pos == 'MID':
        k_G = 6
        k_CS = 0
        k_2GC = 0
    else:
        k_G = 5
        k_CS = 0
        k_2GC = 0
    k_Start = 2
    k_Sub = 1
    k_A = 3
    k_PenSv = 5
    k_PenMiss = -3
    k_Yc = -1
    k_Rc = -3
    k_OG = -2
    k_T1 = 2
    k_T2 = 3
    x_90s = xMins/90
    p_start = 0.5+np.cbrt((x_90s-0.5)/4)
    StartxP = k_Start * p_start
    SubxP = k_Sub * (1-p_start) * x_90s
    GxP = k_G * player_data.loc[0, 'bl_npgp90'] * opp_data.loc[0, 'def_k'] * x_90s * home_adv
    AxP = k_A * player_data.loc[0, 'bl_ap90'] * opp_data.loc[0, 'def_k'] * x_90s * home_adv
    OGxP = k_OG * player_data.loc[0, 'bl_og'] * opp_data.loc[0, 'og_for_k'] * x_90s
    PenScorexP = k_G * player_data.loc[0, 'on_pens'] * player_data.loc[0, 'bl_penscore'] * own_team_data.loc[0, 'pens_p90'] * (opp_data.loc[0, 'pen_against_k'] / opp_data.loc[0, 'pen_save_k']) * x_90s * home_adv
    PenMissxP = k_PenMiss * player_data.loc[0, 'on_pens'] * (1-player_data.loc[0, 'bl_penscore']) * own_team_data.loc[0, 'pens_p90'] * opp_data.loc[0, 'pen_against_k'] * opp_data.loc[0, 'pen_save_k'] * x_90s * home_adv
    if Pos == 'GK':
        PenSvxP = k_PenSv * own_team_data.loc[0, 'pen_against_k'] * player_data.loc[0, 'bl_pensave'] * opp_data.loc[0, 'pens_p90'] * opp_data.loc[0, 'pen_score_k'] * x_90s / home_adv
        mu_sv = player_data.loc[0, 'savep90_2223'] * opp_data.loc[0, 'saves_against_k'] * x_90s / home_adv
        T1SvxP = k_T1 * (poisson.cdf(k=4, mu=mu_sv) - poisson.cdf(k=2, mu=mu_sv))
        T2SvxP = k_T2 * (1 - poisson.cdf(k=4, mu=mu_sv))
    else:
        PenSvxP = T1SvxP = T2SvxP = 0
    YcxP = k_Yc * player_data.loc[0, 'bl_yc'] * opp_data.loc[0, 'yc_against_k'] * x_90s
    RcxP = k_Rc * player_data.loc[0, 'bl_rc'] * opp_data.loc[0, 'yc_against_k'] * x_90s
    mu_gc = own_team_data.loc[0, 'rel_def'] * opp_data.loc[0, 'off_k'] * x_90s  / home_adv
    CSxP = k_CS * poisson.cdf(k=0, mu=mu_gc) * p_start
    GCxP = k_2GC * (1 - poisson.cdf(k=1, mu=mu_gc))
    mu_tack = player_data.loc[0, 'bl_tackp90'] * opp_data.loc[0, 'saves_against_k'] * x_90s
    T1TackxP = k_T1 * (poisson.cdf(k=4, mu=mu_tack) - poisson.cdf(k=3, mu=mu_tack))
    T2TackxP = k_T2 * (1 - poisson.cdf(k=4, mu=mu_tack))
    av_pass = player_data.loc[0, 'bl_passp90'] * opp_data.loc[0, 'pass_against_k'] * x_90s * home_adv
    T1PassxP = k_T1 * (norm.cdf(x=69, loc=av_pass, scale=av_pass/3.5) - norm.cdf(x=59, loc=av_pass, scale=av_pass/3.5))
    T2PassxP = k_T2 * (1 - norm.cdf(x=69, loc=av_pass, scale=av_pass/3.5))
    mu_sot = player_data.loc[0, 'bl_sotp90'] * opp_data.loc[0, 'sot_against_k'] * x_90s * home_adv
    T1SOTxP = k_T1 * (poisson.cdf(k=2, mu=mu_sot) - poisson.cdf(k=1, mu=mu_sot))
    T2SOTxP = k_T2 * (1 - poisson.cdf(k=2, mu=mu_sot))
    xP = GxP + AxP + OGxP + PenScorexP + PenMissxP + PenSvxP + StartxP + SubxP + YcxP + RcxP + CSxP + GCxP + T1SvxP + T1TackxP + T1PassxP + T1SOTxP + T2SvxP + T2TackxP + T2PassxP + T2SOTxP
    if xP_breakdown == True:
        xP_breakdown = {'Actions': ['Start', 'Sub', 'Goal', 'Assist', 'Own Goal', 'Pen Goal', 'Pen Miss', 'Pen Save', 'Yellow Card', 'Red Card', 'Clean Sheet', 'Goal Conceded',
                                    'Tier 1 Save', 'Tier 1 Tackle', 'Tier 1 Pass', 'Tier 1 SOT', 'Tier 2 Save', 'Tier 2 Tackle', 'Tier 2 Pass', 'Tier 2 SOT', 'TOTAL'],
                        'xP': [StartxP, SubxP, GxP, AxP, OGxP, PenScorexP, PenMissxP, PenSvxP, YcxP, RcxP, CSxP, GCxP, T1SvxP, T1TackxP, T1PassxP, T1SOTxP, T2SvxP, T2TackxP, T2PassxP, T2SOTxP, xP]
                        }
        xP_breakdown = pd.DataFrame(xP_breakdown)
        xP_breakdown = xP_breakdown.drop(xP_breakdown[xP_breakdown.xP == 0].index)
        xP_breakdown.xP = round(xP_breakdown.xP,2)
    return {'xP': xP, 'xP_breakdown': xP_breakdown}

def generate_model_output(first_md=1, last_md=14, filename_suffix=None, custom_fixtures=None):
    schedule_name = 'sky_schedule'
    if filename_suffix is not None:
        schedule_name += filename_suffix
    
    if custom_fixtures is not None:
        r = generate_ticker(custom_fixtures=custom_fixtures)
    else:
        r = generate_ticker()
    md_map_2 = r['matchday_map']
    sky_schedule_2 = r['fixtures_data']
    formatted_fixtures = r['formatted_fixtures']
    headers = []
    for i, x in enumerate(sky_schedule_2.columns.values.tolist()):
        if x in md_map_2['unique_dates'].tolist():
            h = md_map_2.loc[md_map_2['unique_dates'] == x, 'matchday'].values[0]
            h = 'MD ' + str(h)
            headers.append(h)
        else:
            headers.append(x)
    sky_schedule_2.columns = headers

    if str(last_md) == last_md:
        if last_md > md_map_2['unique_dates'].tolist()[-1]:
            last_md = md_map_2.loc[len(md_map_2)-1, 'matchday']
        else:
            for i, x in enumerate(md_map_2['unique_dates'].tolist()):
                if last_md < x:
                    last_md = md_map_2.loc[i, 'matchday'] - 1
                    break

    matchdays = range(first_md,last_md+1)
    team_data = prior_team_data_gen(use_weerogue_ts=1)
    player_data = prior_player_data_gen(team_data)
    prior_player_data = player_data['prior_player_data']

    fpd = pd.merge(prior_player_data, sky_schedule_2, left_on='team_name', right_on='short_name', how='left')
    fixture_player_data = fpd.copy()
    for i in range(first_md, last_md+1):
        gw = md_map_2.loc[md_map_2['matchday']==i, 'fpl_gw'].values[0]
        if f'{gw}_xMins' not in fixture_player_data.columns:
            fixture_player_data[f'{gw}_xMins'] = fixture_player_data[f'{gw-1}_xMins']
        fixture_player_data[f'MD {i} Game'] = fixture_player_data[f'MD {i}'].str.len() > 1.5
        fixture_player_data[f'MD_{i}_xMins'] = fixture_player_data[f'{gw}_xMins'] * fixture_player_data[f'MD {i} Game']
    players = fixture_player_data.index.tolist()
    for p in players:
        SKY_ID = fixture_player_data.loc[p, 'SKY ID']
        for m in matchdays:
            xMins = fixture_player_data.loc[p, f'MD_{m}_xMins']
            if xMins < 5:
                xP = 0
            else:
                fix_string = fixture_player_data.loc[p, f'MD {m}']
                if '\n' in fix_string:
                    xP = 0
                    fix_list = fix_string.split('\n')
                    for i, x in enumerate(fix_list):
                        r = sky_xP_calc(SKY_ID, x[:3], fixture_player_data, team_data, xMins, xP_breakdown=False)
                        sub_xP = r['xP']
                        if any(c.isdigit() for c in x):
                            sub_xP = sub_xP * int(''.join(filter(str.isdigit, x))) / 100
                        xP += sub_xP
                else:
                    r = sky_xP_calc(SKY_ID, fixture_player_data.loc[p, f'MD {m}'][:3], fixture_player_data, team_data, xMins, xP_breakdown=False)
                    xP = r['xP']
                    if any(c.isdigit() for c in fix_string):
                        xP = xP * int(''.join(filter(str.isdigit, fix_string))) / 100
            fixture_player_data.loc[p, f'MD_{m}_Pts'] = round(xP, 2)
    skymodel_output = pd.concat([fixture_player_data.loc[:,['SKY ID', 'Name', 'team_name', 'SKY Pos', 'SKY Value']],
                                    fixture_player_data.iloc[:,-(last_md-first_md+1):]],axis = 1)
    skymodel_output['Total_Pts'] = skymodel_output.iloc[:, -(last_md-first_md+1):].sum(axis=1)
    skymodel_output = skymodel_output.fillna(0)
    filename = 'skymodel_output'
    if filename_suffix is not None:
        filename += filename_suffix
    skymodel_output.to_csv(f'../data/{filename}.csv')

    return {'skymodel_output':skymodel_output, 'formatted_fixtures':formatted_fixtures, 'md_map': md_map_2}

## Optimization Functions

Generate optimal solution for team or analyze multiple simulated runs with noise, based on the model output.

In [148]:
data = {'SKY Pos': ['GK', 'DEF', 'MID', 'FOR'],
        'squad_min_play': [1, 3, 3, 1],
        'squad_max_play': [1, 5 ,5, 3]}
type_data = pd.DataFrame(data, index=[1,2,3,4])

def solve_sky_mp(initial_squad, input_data, md_map, next_md=1, last_md=10, 
                 ta_tot=50, ta_gw=5, objective='regular', decay_base=0.85, transfer_cost=7.5, 
                 exclusions=None, keeps=None, force_transfer_in=None, force_transfer_out=None,
                 apply_noise=False, seed_val=None, magnitude=1):
    
    if str(last_md) == last_md:
        if last_md > md_map['unique_dates'].tolist()[-1]:
            last_md = md_map.loc[len(md_map)-1, 'matchday']
        else:
            for i, x in enumerate(md_map['unique_dates'].tolist()):
                if last_md < x:
                    last_md = md_map.loc[i, 'matchday'] - 1
                    break
    horizon = last_md + 1 - next_md
    problem_name = f'sky_mp_h{horizon}_d1'
    
    # Sets
    players = input_data.index.tolist()
    element_types = type_data.index.tolist()
    matchdays = list(range(next_md, next_md+horizon))
    all_md = [next_md-1] + matchdays
    
    first_gw = int(md_map.loc[md_map['matchday']==next_md, 'sky_gw'].values[0])
    last_gw = int(md_map.loc[md_map['matchday']==last_md, 'sky_gw'].values[0])
    gameweeks = list(range(first_gw,last_gw+1))
    gw_transfer_allowance = {w: 5 for w in gameweeks}
    gw_transfer_allowance[first_gw] = min(ta_gw,5)

    if apply_noise:
        rng = np.random.default_rng(seed = seed_val)
        input_data['Total_Pts'] = 0
        for m in matchdays:
            noise = input_data[f'MD_{m}_Pts'] * (91.6 - 80) / 134.1 * rng.standard_normal(size = len(input_data)) * magnitude
            input_data[f'MD_{m}_Pts'] = input_data[f'MD_{m}_Pts'] + round(noise,2)
            input_data['Total_Pts'] += input_data[f'MD_{m}_Pts']

    # Model
    model = so.Model(name = 'multi_period');

    # Variables
    squad = model.add_variables(players, all_md, name='squad', vartype=so.binary)
    captain = model.add_variables(players, matchdays, name='captain', vartype=so.binary)
    transfer_in = model.add_variables(players, matchdays, name='transfer_in', vartype=so.binary)
    transfer_out = model.add_variables(players, matchdays, name='transfer_out', vartype=so.binary)
    
    # Dictionaries
    squad_type_count = {(t,d): so.expr_sum(squad[p,d] for p in players if input_data.loc[p, 'SKY Pos'] == type_data.loc[t, 'SKY Pos']) for t in element_types for d in matchdays}
    player_value = (input_data['SKY Value']).to_dict()
    bought_amount = {d: so.expr_sum(player_value[p] * transfer_in[p,d] for p in players) for d in matchdays}
    sold_amount = {d: so.expr_sum(player_value[p] * transfer_out[p,d] for p in players) for d in matchdays}
    squad_value = {d: so.expr_sum(player_value[p] * squad[p,d] for p in players) for d in matchdays}
    points_player_day = {(p,d): input_data.loc[p, f'MD_{d}_Pts'] for p in players for d in matchdays}
    squad_count = {d: so.expr_sum(squad[p, d] for p in players) for d in matchdays}
    
    total_number_of_transfers = so.expr_sum(transfer_out[p,d] for p in players for d in matchdays) 

    md_number_of_transfers = {d: so.expr_sum(transfer_out[p,d] for p in players) for d in matchdays}        
    gw_number_of_transfers = {w: so.expr_sum(md_number_of_transfers[d] for d in matchdays if int(md_map.loc[md_map['matchday']==d, 'sky_gw'].values[0]) == w) for w in gameweeks}
    
    # Initial Conditions
    if initial_squad is not None:
        model.add_constraints((squad[p, next_md-1] == 1 for p in initial_squad), name='initial_squad_players')
        model.add_constraints((squad[p, next_md-1] == 0 for p in players if p not in initial_squad), name='initial_squad_others')
    # Constraints: squad and captaincy
    model.add_constraints((squad_count[d] == 11 for d in matchdays), name='squad_count')
    model.add_constraints((so.expr_sum(captain[p,d] for p in players) == 1 for d in matchdays), name='captain_count')
    model.add_constraints((captain[p,d] <= squad[p,d] for p in players for d in matchdays), name='captain_squad_rel')
    # Constraints: formation and budget
    model.add_constraints((squad_type_count[t,d] == [type_data.loc[t, 'squad_min_play'], type_data.loc[t, 'squad_max_play']] for t in element_types for d in matchdays), name='valid_formation_1')
    model.add_constraints((squad_type_count[2,d]-squad_type_count[4,d] <= 3.5 for d in matchdays), name='valid_formation_2')
    model.add_constraints((squad_value[d] <= 100 for d in matchdays), name='squad_budget')
    # Constraints: generic transfers
    model.add_constraints((squad[p,d] == squad[p,d-1] + transfer_in[p,d] - transfer_out[p,d] for p in players for d in matchdays), name='squad_transfer_rel')
    model.add_constraint(total_number_of_transfers <= min(ta_tot,50), name = 'transfer_allowance')
    model.add_constraints((gw_number_of_transfers[w] <= gw_transfer_allowance[w] for w in gameweeks), name = 'gw_transfer_allowance')
    # Constraints: specified players
    # Force Exclude
    if exclusions is not None:
        model.add_constraints((squad[e, d] == 0 for e in exclusions for d in matchdays), name = 'force_exclude_players')
    # Force Keep
    if keeps is not None:
        model.add_constraints((squad[e, d] == 1 for e in keeps for d in matchdays), name = 'force_keep_players')
    # Force transfer in
    if force_transfer_in is not None:
        model.add_constraints((squad[force_transfer_in[e][0], force_transfer_in[e][1]] == 1 for e in list(range(len(force_transfer_in)))), name = 'force_transfer_in_players')
        model.add_constraints((squad[force_transfer_in[e][0], force_transfer_in[e][1]-1] == 0 for e in list(range(len(force_transfer_in)))), name = 'force_transfer_in_players_2')
    # Force transfer out
    if force_transfer_out is not None:
        model.add_constraints((squad[force_transfer_out[e][0], force_transfer_out[e][1]] == 0 for e in list(range(len(force_transfer_out)))), name = 'force_transfer_out_players')
        model.add_constraints((squad[force_transfer_out[e][0], force_transfer_out[e][1]-1] == 1 for e in list(range(len(force_transfer_out)))), name = 'force_transfer_out_players_2')
    
    # Objective
    md_xp = {d: so.expr_sum(points_player_day[p,d] * (squad[p,d] + captain[p,d]) for p in players) for d in matchdays}
    if objective == 'regular':
        total_xp = so.expr_sum(md_xp[d] for d in matchdays) - total_number_of_transfers*transfer_cost
        model.set_objective(-total_xp, sense='N', name='total_regular_xp') 
    else:
        # total_xp = so.expr_sum(md_xp[d] * pow(decay_base, d-next_md) for d in matchdays) - total_number_of_transfers*transfer_cost
        days_elapsed0 = md_map.loc[md_map['matchday']==next_md,'days_elapsed'].values[0]
        # Convert weekly decay to daily
        decay_base = decay_base ** (1/7)
        total_xp = so.expr_sum(md_xp[d] * pow(decay_base, md_map.loc[md_map['matchday']==d,'days_elapsed'].values[0]-days_elapsed0) for d in matchdays) - total_number_of_transfers*transfer_cost
        model.set_objective(-total_xp, sense='N', name='total_decay_xp')
    
    # Solve Step
    model.export_mps(filename='skyoutput.mps')
    command = f'cbc skyoutput.mps solve solu {problem_name}_sol.txt'
    # !{command}
    os.system(command)
    # Read the solution back to the file
    with open(f'{problem_name}_sol.txt', 'r') as f:
        for v in model.get_variables():
            v.set_value(0)
        for line in f:
            if 'objective value' in line:
                continue
            words = line.split()
            var = model.get_variable(words[1])
            var.set_value(float(words[2]))
            
    # (OLD) Generate a dataframe to display the solution 
    picks = []
    for d in matchdays:
        for p in players:
            if squad[p,d].get_value() + transfer_out[p,d].get_value() > 0.5:
                lp = input_data.loc[p]
                is_captain = 1 if captain[p,d].get_value() > 0.5 else 0
                is_transfer_in = 1 if transfer_in[p,d].get_value() > 0.5 else 0
                is_transfer_out = 1 if transfer_out[p,d].get_value() > 0.5 else 0
                picks.append([
                    int(md_map.loc[md_map['matchday']==d, 'sky_gw'].values[0]), d, lp['Name'], lp['SKY Pos'], lp['team_name'], lp['SKY Value'], round(points_player_day[p,d], 2), is_captain, is_transfer_in, is_transfer_out
                ])
    picks_df = pd.DataFrame(picks, columns=['sky_gw','matchday','name', 'pos', 'team', 'value', 'xP', 'captain', 'transfer_in', 'transfer_out'])#.sort_values(by=['matchday'])
    picks_df.loc[picks_df['matchday'] == next_md, 'transfer_in'] = 0
    
    total_xp = round(so.expr_sum(points_player_day[p,d] * (squad[p,d] + captain[p,d]) for p in players for d in matchdays).get_value(), 2)
    
    # Generate a better dataframe to display the solution
    plan = []
    for t in element_types:
        for p in players:
            if so.expr_sum(squad[p,d] + transfer_out[p,d] for d in matchdays).get_value() >= 0.5 and input_data.loc[p, 'SKY Pos'] == type_data.loc[t, 'SKY Pos']:
                lp = input_data.loc[p]
                player_info = [p, lp['team_name'], lp['SKY Pos'], lp['SKY Value'], lp['Name']]
                for d in matchdays:
                    if squad[p,d].get_value() > 0.5:
                        score = f'{round(points_player_day[p,d], 2)}'
                        if captain[p,d].get_value() > 0.5:
                            score += 'c'
                    else:
                        score = ''
                    player_info.append(score)
                plan.append(player_info)
    columns = ['ID','Team', 'Pos','Value','Name']
    for d in matchdays:
        # w = int(md_map.loc[md_map['matchday']==d, 'sky_gw'].values[0])
        columns.append(f"{d}")
    plan_df = pd.DataFrame(plan, columns=columns)
    plan_df = plan_df.replace(['0.0'],'-')
    plan_df = plan_df.replace(['0.0c'],'-')

    # make dataframe to record the players in a simulation
    plan = []
    if apply_noise:
        for t in element_types:
            for p in players:
                if so.expr_sum(squad[p,d] + transfer_out[p,d] for d in matchdays).get_value() >= 0.5 and input_data.loc[p, 'SKY Pos'] == type_data.loc[t, 'SKY Pos']:
                    lp = input_data.loc[p]
                    player_info = [p, lp['team_name'], lp['SKY Pos'], lp['SKY Value'], lp['Name']]
                    for d in matchdays:
                        if squad[p,d].get_value() > 0.5:
                            score = 1
                        else:
                            score = 0
                        player_info.append(score)
                    plan.append(player_info)
        columns = ['ID','Team', 'Pos','Value','Name']
        for d in matchdays:
            # w = int(md_map.loc[md_map['matchday']==d, 'sky_gw'].values[0])
            columns.append(f"{d}")
        players_in_sim = pd.DataFrame(plan, columns=columns)
    else:
        players_in_sim = None
    
    sky_gw_header = []
    fpl_gw_header = []
    for i in columns:
        if i == 'Name':
            j = 'sky_gw'
            k = 'fpl_gw'
        elif not str(i)[0].isdigit():
            j = ''
            k = ''
        else:
            sky_gw = md_map.loc[md_map['matchday']==int(i),'sky_gw'].values[0]
            fpl_gw = md_map.loc[md_map['matchday']==int(i),'fpl_gw'].values[0]
            j = str(sky_gw)
            k = str(fpl_gw)
        sky_gw_header.append(j)
        fpl_gw_header.append(k)
    plan_df.columns=[fpl_gw_header, sky_gw_header, columns]

    transfers_made = int(total_number_of_transfers.get_value())
    
    return{'model': model, 'picks': picks_df, 'total_xp': total_xp, 'plan': plan_df, 'transfers_made': transfers_made, 'players_in_sim': players_in_sim}

# Produce sensitivity analysis with noise
def solve_sky_mp_noise(initial_squad, input_data, md_map, next_md=1, last_md=10, 
                       ta_tot=50, ta_gw=5, objective='regular', decay_base=0.85, transfer_cost=7.5, 
                       exclusions=None, keeps=None, force_transfer_in=None, force_transfer_out=None,
                       seed_val=None, nsims=5, magnitude=1):
    transfer_sum = 0

    for i in range(nsims):
        results = solve_sky_mp(initial_squad=initial_squad, input_data=input_data, md_map=md_map, next_md=next_md, last_md=last_md, 
                    ta_tot=ta_tot, ta_gw=ta_gw, objective=objective, decay_base=decay_base, transfer_cost=transfer_cost,
                    exclusions=exclusions, keeps=keeps, force_transfer_in=force_transfer_in, force_transfer_out=force_transfer_out,
                    apply_noise=True, seed_val=seed_val, magnitude=magnitude)
        players_in_sim = results['players_in_sim']
        # display(players_in_sim)
        if i == 0:
            # display(players_in_sim)
            sensitivity_df = players_in_sim
        else:
            for index, row in results['players_in_sim'].iterrows():
                if row['ID'] in sensitivity_df['ID'].tolist():
                    sensitivity_df.loc[sensitivity_df['ID']==row['ID'], '1':] = sensitivity_df.loc[sensitivity_df['ID']==row['ID'], '1':] + row['1':]
                    continue
                else: 
                    sensitivity_df = sensitivity_df.append(row, ignore_index=True)
        transfer_sum += results['transfers_made']
    avg_trf = transfer_sum/nsims

    sensitivity_df.loc[:,'1':] = sensitivity_df.loc[:,'1':] * 100 / nsims
    sensitivity_df.loc[:,'1':] = sensitivity_df.loc[:,'1':].astype(int)

    columns = sensitivity_df.columns.values.tolist()

    sky_gw_header = []
    fpl_gw_header = []
    for i in columns:
        if i == 'Name':
            j = 'sky_gw'
            k = 'fpl_gw'
        elif not str(i)[0].isdigit():
            j = ''
            k = ''
        else:
            sky_gw = md_map.loc[md_map['matchday']==int(i),'sky_gw'].values[0]
            fpl_gw = md_map.loc[md_map['matchday']==int(i),'fpl_gw'].values[0]
            j = str(sky_gw)
            k = str(fpl_gw)
        sky_gw_header.append(j)
        fpl_gw_header.append(k)
    sensitivity_df.columns=[fpl_gw_header, sky_gw_header, columns]

    sens = sensitivity_df.style.background_gradient(cmap="RdPu", subset=sensitivity_df.columns[5:]).format(precision=1)
    # sens.set_properties(**{'text-align': 'left'})
    sens = sens.set_table_styles([
                        {'selector': 'th.col_heading', 'props': 'text-align: left;'},
                        {'selector': 'th.col_heading.level0', 'props': 'font-size: 1em;'},
                        {'selector': 'td', 'props': 'text-align: center; font-weight: bold;'},
                    ], overwrite=False)
                
    return {'sensitivity_df': sens, 'avg_trf': avg_trf}

## My Commands

In [31]:
# Note: make sure to upload fplreview.csv or similarly formatted xMins to data folder

# Example custom fixtures dataframe
df = pd.DataFrame(columns=('home_team', 'away_team', 'dates', 'probabilities'))
df.loc[len(df)] = ['NEW', 'BHA', ['2023-04-18','2023-05-09'], [0.5, 0.5]]
df.loc[len(df)] = ['BHA', 'MCI', ['2023-05-02'], [0]]
df.loc[len(df)] = ['FUL', 'LEE', ['2023-04-22'], [1]]
df.loc[len(df)] = ['MUN', 'CHE', ['2023-04-22'], [0]]
df.loc[len(df)] = ['LIV', 'FUL', ['2023-05-02'], [1]]
df.loc[len(df)] = ['MCI', 'WHU', ['2023-05-02'], [1]]
df.loc[len(df)] = ['BHA', 'MUN', ['2023-05-02'], [1]]

# Generate fixture ticker and player EV
r = generate_model_output(first_md=1, last_md='2023-05-13', filename_suffix=None, custom_fixtures=df)
md_map = r['md_map']
display(r['formatted_fixtures'])
display(r['skymodel_output'].sort_values(by=['Total_Pts'], ascending=False).head(20))

,SKY ID,Name,team_name,SKY Pos,SKY Value,MD_1_Pts,MD_2_Pts,MD_3_Pts,MD_4_Pts,MD_5_Pts,MD_6_Pts,MD_7_Pts,MD_8_Pts,MD_9_Pts,MD_10_Pts,MD_11_Pts,MD_12_Pts,MD_13_Pts,MD_14_Pts,MD_15_Pts,MD_16_Pts,MD_17_Pts,MD_18_Pts,MD_19_Pts,MD_20_Pts,MD_21_Pts,MD_22_Pts,MD_23_Pts,MD_24_Pts,MD_25_Pts,MD_26_Pts,MD_27_Pts,MD_28_Pts,MD_29_Pts,MD_30_Pts,Total_Pts
232,872,Raya,BRE,GK,6.7,5.92,0.00,5.91,0.00,5.56,0.00,5.30,0.00,0.00,0.00,5.15,5.53,0.00,6.09,0.00,0.00,0.00,0.0,5.83,0.00,0.00,5.09,0.00,6.02,0.00,0.0,0.00,4.67,0.00,5.72,66.79
510,1280,Haaland,MCI,FOR,11.5,6.64,0.00,0.00,0.00,0.00,0.00,7.09,0.00,0.00,0.00,0.00,0.00,6.68,0.00,8.03,0.00,0.00,0.0,0.00,0.00,0.00,5.87,0.00,0.00,6.49,0.0,6.80,7.89,0.00,6.61,62.10
368,121,Pope,NEW,GK,7.1,0.00,5.92,0.00,5.39,0.00,0.00,0.00,4.88,0.00,0.00,5.06,4.75,0.00,5.15,0.00,0.00,2.52,0.0,0.00,4.58,0.00,0.00,5.44,0.00,5.73,0.0,0.00,4.36,2.52,4.88,61.18
262,100,Dunk,BHA,DEF,7.9,4.09,0.00,6.70,0.00,0.00,0.00,6.02,0.00,0.00,5.94,0.00,4.09,0.00,3.35,0.00,0.00,1.70,0.0,0.00,0.00,0.00,5.73,0.00,6.48,0.00,0.0,5.25,6.51,1.69,2.70,60.25
438,351,Salah,LIV,MID,11.7,6.19,0.00,0.00,0.00,0.00,0.00,3.81,0.00,0.00,4.59,0.00,0.00,4.81,0.00,0.00,5.52,0.00,0.0,6.64,0.00,0.00,4.85,0.00,0.00,5.62,0.0,6.29,6.28,0.00,5.31,59.91
440,337,Alexander-Arnold,LIV,DEF,10.8,6.70,0.00,0.00,0.00,0.00,0.00,2.56,0.00,0.00,3.80,0.00,0.00,4.45,0.00,0.00,4.53,0.00,0.0,7.77,0.00,0.00,5.51,0.00,0.00,6.04,0.0,6.65,6.57,0.00,5.16,59.74
172,148,Arrizabalaga,CHE,GK,6.6,5.70,0.00,0.00,0.00,6.49,0.00,6.19,0.00,0.00,5.13,0.00,6.40,0.00,5.84,0.00,0.00,0.00,0.0,0.00,0.00,0.00,5.76,0.00,4.94,0.00,0.0,0.00,6.05,0.00,6.43,58.93
523,548,Kane,TOT,FOR,11.6,7.41,0.00,0.00,0.00,5.95,0.00,0.00,0.00,5.89,0.00,0.00,5.39,0.00,7.16,0.00,0.00,0.00,0.0,0.00,4.33,0.00,0.00,5.79,0.00,4.95,0.0,0.00,6.18,0.00,5.07,58.12
436,347,Alisson,LIV,GK,8.0,5.71,0.00,0.00,0.00,0.00,0.00,4.08,0.00,0.00,5.04,0.00,0.00,4.76,0.00,0.00,5.31,0.00,0.0,6.18,0.00,0.00,5.49,0.00,0.00,5.06,0.0,5.58,5.42,0.00,5.31,57.94
351,1175,Trippier,NEW,DEF,7.8,0.00,6.30,0.00,5.27,0.00,0.00,0.00,4.97,0.00,0.00,4.61,4.11,0.00,4.27,0.00,0.00,2.26,0.0,0.00,5.17,0.00,0.00,4.69,0.00,5.94,0.0,0.00,3.81,2.22,4.14,57.76


In [34]:
skymodel_output = pd.read_csv('../data/skymodel_output.csv').set_index('SKY ID').fillna(0)

# Sky IDs of all players to include in the initial squad
team = [872,
377,13,20,100,337,
351,810,895,
548,1275]
transfer_cost = 0

# Generate optimal solution
r2 = solve_sky_mp(initial_squad=team, input_data=skymodel_output, md_map=md_map, next_md=1, last_md='2023-04-09', 
                 ta_tot=5, ta_gw=5, objective='decay', decay_base=1, transfer_cost=transfer_cost,
                 exclusions=None, keeps=None, force_transfer_in=None, force_transfer_out=None,
                 apply_noise=True)
display(r2['plan'], f"Total xP: {r2['total_xp']}", f"Total transfers made: {r2['transfers_made']}, transfer cost: {transfer_cost}")

NOTE: Initialized model multi_period.


fpl_gw     27                  28         \
                                    sky_gw     32                  33          
      ID Team  Pos Value              Name      1      2     3      4      5   
0    872  BRE   GK   6.7              Raya   6.41      -   6.6      -   5.51   
1     20  ARS  DEF   9.1           Gabriel      -   4.11     -      -      -   
2     13  ARS  DEF   7.4            Saliba      -   4.02     -      -      -   
3    377  ARS  DEF   8.0         Zinchenko      -   5.37     -      -      -   
4    100  BHA  DEF   7.9              Dunk   4.81      -  6.9c      -      -   
5    425  MUN  DEF   7.9              Shaw                                     
6   1364  MUN  DEF   8.3          Martínez                                     
7   1175  NEW  DEF   7.8          Trippier         6.94c     -  5.73c      -   
8    337  LIV  DEF  10.8  Alexander-Arnold   6.67                              
9    810  ARS  MID   8.2          Ødegaard      -   4.28     -      -      -   
10   895  BRE  MID   7.7            Mbeumo   4.56      -  3.98      -    4.5   
11   417  MUN  MID   8.1          Rashford                                     
12   351  LIV  MID  11.7             Salah   5.04      -     -      -      -   
13   645  AVL  FOR   9.0           Watkins          4.75     -      -   5.07   
14  1275  LIV  FOR  10.5            Darwin   6.95                              
15   548  TOT  FOR  11.6              Kane  7.05c      -     -      -  6.29c   

              29                               30         
              35                               36         
        6      7      8     9     10     11    12     13  
0       -   5.41      -     -      -   5.38  5.01      -  
1    6.57   5.01                                          
2     6.9  5.64c                                          
3   7.23c    5.1                                          
4       -   5.25      -     -  6.38c      -  4.05      -  
5                   2.8     -      -   5.66  6.05      -  
6                  3.23     -      -   5.01  5.01      -  
7       -      -  5.67c     -      -   5.15   4.1      -  
8                                                         
9     4.5   5.29      -     -      -      -     -    3.8  
10      -   3.65      -     -      -   3.93  3.11      -  
11                 3.42     -      -  5.88c  6.3c      -  
12      -   3.97      -     -   4.61      -     -  5.31c  
13      -   3.42      -     -   4.21      -  4.79      -  
14                                                        
15      -      -      -  6.1c      -      -  4.74      -

'Total xP: 384.6'

'Total transfers made: 5, transfer cost: 0'

In [154]:
skymodel_output = pd.read_csv('../data/skymodel_output.csv').set_index('SKY ID').fillna(0)

# Sky IDs of all players to include in the initial squad
team = [872,
377,13,20,100,337,
351,810,895,
548,1275]
transfer_cost = 7.5

# Generate sensitivity analysis
r2 = solve_sky_mp_noise(initial_squad=team, input_data=skymodel_output, md_map=md_map, next_md=1, last_md='2023-04-23', 
                        ta_tot=20, ta_gw=5, objective='decay', decay_base=1, transfer_cost=transfer_cost,
                        exclusions=None, keeps=None, force_transfer_in=None, force_transfer_out=None,
                        seed_val=None, nsims=50, magnitude=0.5)
display(r2['sensitivity_df'], f"Average transfers made: {r2['avg_trf']}")

NOTE: Initialized model multi_period.
NOTE: Initialized model multi_period.
NOTE: Initialized model multi_period.
NOTE: Initialized model multi_period.
NOTE: Initialized model multi_period.
NOTE: Initialized model multi_period.
NOTE: Initialized model multi_period.
NOTE: Initialized model multi_period.
NOTE: Initialized model multi_period.
NOTE: Initialized model multi_period.
NOTE: Initialized model multi_period.
NOTE: Initialized model multi_period.
NOTE: Initialized model multi_period.
NOTE: Initialized model multi_period.
NOTE: Initialized model multi_period.
NOTE: Initialized model multi_period.
NOTE: Initialized model multi_period.
NOTE: Initialized model multi_period.
NOTE: Initialized model multi_period.
NOTE: Initialized model multi_period.
NOTE: Initialized model multi_period.
NOTE: Initialized model multi_period.
NOTE: Initialized model multi_period.
NOTE: Initialized model multi_period.
NOTE: Initialized model multi_period.
NOTE: Initialized model multi_period.
NOTE: Initia

'Average transfers made: 17.56'